In [0]:
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 39kB/s 
     |████████████████████████████████| 4.3MB 42.3MB/s 
     |████████████████████████████████| 501kB 48.0MB/s 


In [0]:
!pip install tf-nightly-gpu

In [0]:
# mount to your google drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import numpy as np
import math
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0-dev20200307


In [0]:
#음수의 값을 표현하기 위함.
def swish_activation(x):
  return x * tf.nn.sigmoid(x)

#width 조건 설정.
def round_filters(filters, multiplier):
    depth_divisor = 8
    min_depth = None
    min_depth = min_depth or depth_divisor
    filters = filters * multiplier
    new_filters = max(min_depth, int(filters + depth_divisor / 2) // depth_divisor * depth_divisor)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)

#depth 조건 설정.
def round_repeats(repeats, multiplier):
    if not multiplier:
        return repeats
    return int(math.ceil(multiplier * repeats))


In [0]:
class EfficientNet_classification(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(EfficientNet_classification, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

In [0]:
class MBConv(tf.keras.layers.Layer):
  def __init__(self, kernel_size, input_filters, output_filters, \
                              expand_ratio, stride, id_skip, se_ratio):
    super(MBConv, self).__init__()
    '''Organized separately from active call parts.'''
    #print('kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio :', \
    #      kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio)
    # expansion phase
    self.conv_1 = tf.keras.layers.Conv2D(filters = input_filters*expand_ratio, 
                                          kernel_size=(1, 1),  
                                          padding='same')
    self.bn_1 = tf.keras.layers.BatchNormalization()

    # Depthwise convolution phase
    self.dw_conv_1 = tf.keras.layers.DepthwiseConv2D(kernel_size = kernel_size, 
                                                     strides = stride,  
                                                     padding='same')
    self.bn_2 = tf.keras.layers.BatchNormalization()

    # Squeeze and excitation phase
    self.se_1 = tf.keras.layers.GlobalAveragePooling2D()
    self.se_2 = tf.keras.layers.Reshape((1, 1, input_filters*expand_ratio ))
    self.squeezed_filters = max (1, int(input_filters * se_ratio))

    self.se_3 = tf.keras.layers.Conv2D(filters = self.squeezed_filters, 
                                       kernel_size=(1, 1), 
                                       padding='same')
    self.se_4 = tf.keras.layers.Conv2D(filters = input_filters*expand_ratio, 
                                       kernel_size=(1, 1), 
                                       padding='same')
    # Output phase
    self.conv_2 = tf.keras.layers.Conv2D(filters = output_filters, 
                                         kernel_size=(1, 1), 
                                         padding='same')
    self.bn_3 = tf.keras.layers.BatchNormalization()

  def call(self, inputs):
    # expansion phase
    x = self.conv_1(inputs)
    x = self.bn_1(x)
    x = swish_activation(x)
    # Depthwise convolution phase
    x = self.dw_conv_1(x)
    x = self.bn_2(x)
    x = swish_activation(x)
    # Squeeze and excitation phase
    se = self.se_1(x)
    se = self.se_2(x)
    se = self.se_3(x)
    se = swish_activation(x)
    se = self.se_4(x)
    se = tf.nn.sigmoid(x)
    x = tf.keras.layers.multiply([x, se])
    # Output phase
    x = self.conv_2(x)
    x = self.bn_3(x)
    
    return x

In [0]:
def build_MBConv(kernel_size, num_repeat, input_filters, output_filters, \
                 expand_ratio, stride, id_skip=True, se_ratio=0.25, depth=None, width=None):
  mbConv_block = tf.keras.Sequential()
  depth_layer = round_repeats(num_repeat, depth)
  for i in range(depth_layer):
    if i == 0:
      input_filter = round_repeats(input_filters, width)
      output_filter = round_repeats(output_filters, width)
      mbConv_block.add(MBConv(kernel_size, input_filter, output_filter, \
                              expand_ratio, stride, id_skip, se_ratio))
    else:
      input_filter = round_repeats(input_filters, width)
      output_filter = round_repeats(output_filters, width)
      mbConv_block.add(MBConv(kernel_size, input_filter, output_filter, \
                              expand_ratio, 1, id_skip, se_ratio))
  return mbConv_block

In [0]:
class EfficientNet(tf.keras.Model):
  def __init__(self, width, depth, pool='avg', classes=1000, include_top=True):
    super(EfficientNet, self).__init__()

    self.conv_1 = tf.keras.layers.Conv2D(filters=round_filters(32, width),
                                         kernel_size=(3, 3),
                                         strides=2,
                                         padding="same")
    self.bn_1 = tf.keras.layers.BatchNormalization()

    self.MBConv_1 = build_MBConv(kernel_size=(3, 3), num_repeat=1, input_filters=32, \
                                 output_filters=16, expand_ratio=1, stride=1, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_2 = build_MBConv(kernel_size=(3, 3), num_repeat=2, input_filters=16, \
                                 output_filters=24, expand_ratio=6, stride=2, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_3 = build_MBConv(kernel_size=(5, 5), num_repeat=2, input_filters=24, \
                                 output_filters=40, expand_ratio=6, stride=2, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_4 = build_MBConv(kernel_size=(3, 3), num_repeat=3, input_filters=40, \
                                 output_filters=80, expand_ratio=6, stride=2, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_5 = build_MBConv(kernel_size=(5, 5), num_repeat=3, input_filters=80, \
                                 output_filters=112, expand_ratio=6, stride=1, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_6 = build_MBConv(kernel_size=(5, 5), num_repeat=4, input_filters=112, \
                                 output_filters=192, expand_ratio=6, stride=2, id_skip=True, se_ratio=0.25, depth=depth, width=width)
    self.MBConv_7 = build_MBConv(kernel_size=(3, 3), num_repeat=1, input_filters=192, \
                                 output_filters=320, expand_ratio=6, stride=1, id_skip=True, se_ratio=0.25, depth=depth, width=width)

    self.conv_2 = tf.keras.layers.Conv2D(filters=round_filters(1280, width),
                                        kernel_size=(1, 1),
                                        strides=1,
                                        padding="same")
    self.bn_2 = tf.keras.layers.BatchNormalization()
    self.fc = EfficientNet_classification(pooling=pool, classes=classes)
  def call(self, inputs):
    x = self.conv_1(inputs)
    x = self.bn_1(x)
    x = self.MBConv_1(x)
    x = self.MBConv_2(x)
    x = self.MBConv_3(x)
    x = self.MBConv_4(x)
    x = self.MBConv_5(x)
    x = self.MBConv_6(x)
    x = self.MBConv_7(x)
    x = self.conv_2(x)
    x = self.bn_2(x)
    x = self.fc(x)

    return x

kernel_size는 컨볼 루션의 커널 크기입니다. 예 : 3 x 3

num_repeat는 특정 블록을 몇 번 반복해야하는지 지정합니다. 0보다 커야합니다.

input_filters 및 output_filters는 지정된 필터 수입니다.

expand_ratio는 입력 필터 확장 비율입니다.

id_skip는 건너 뛰기 연결 사용 여부를 제안합니다

se_ratio는 압착 및 여기 블록에 대한 압착 비율을 제공합니다

In [0]:
def get_efficient_net(width_coefficient, depth_coefficient, resolution, pool='avg', classes=1000, include_top=True):
    return EfficientNet(width=width_coefficient, depth=depth_coefficient, pool=pool, classes=classes, include_top=include_top)

def efficient_net_b0():
    return get_efficient_net(width_coefficient=1.0, depth_coefficient=1.0, resolution=224, pool='avg', classes=1000, include_top=True)


def efficient_net_b1():
    return get_efficient_net(width_coefficient=1.0, depth_coefficient=1.1, resolution=244, pool='avg', classes=1000, include_top=True)


def efficient_net_b2():
    return get_efficient_net(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, pool='avg', classes=1000, include_top=True)


def efficient_net_b3():
    return get_efficient_net(width_coefficient=1.2, depth_coefficient=1.4, resolution=300, pool='avg', classes=1000, include_top=True)


def efficient_net_b4():
    return get_efficient_net(width_coefficient=1.4, depth_coefficient=1.8, resolution=380, pool='avg', classes=1000, include_top=True)


def efficient_net_b5():
    return get_efficient_net(width_coefficient=1.6, depth_coefficient=2.2, resolution=456, pool='avg', classes=1000, include_top=True)


def efficient_net_b6():
    return get_efficient_net(width_coefficient=1.8, depth_coefficient=2.6, resolution=580, pool='avg', classes=1000, include_top=True)


def efficient_net_b7():
    return get_efficient_net(width_coefficient=2.0, depth_coefficient=3.1, resolution=600, pool='avg', classes=1000, include_top=True)


In [0]:
efficient_b7 = efficient_net_b7()

kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 64 32 1 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 64 32 1 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 64 32 1 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 64 32 1 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 32 48 6 2 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 32 48 6 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 32 48 6 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 32 48 6 1 True 0.25
kernel_size, input_filters, output_filters, expand_ratio, stride, id_skip, se_ratio (3, 3) 32 48 6 1 True 0.25
k

In [0]:
def print_model_summary(network):
    network.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, 3))
    network.summary()

print_model_summary(network=efficient_b7)

Model: "efficient_net_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1347 (Conv2D)         multiple                  1792      
_________________________________________________________________
batch_normalization_1017 (Ba multiple                  256       
_________________________________________________________________
sequential_93 (Sequential)   multiple                  44736     
_________________________________________________________________
sequential_94 (Sequential)   multiple                  423368    
_________________________________________________________________
sequential_95 (Sequential)   multiple                  993604    
_________________________________________________________________
sequential_96 (Sequential)   multiple                  4001800   
_________________________________________________________________
sequential_97 (Sequential)   multiple             